<a href="https://colab.research.google.com/github/paulaleite/aprendizagemDeMaquinaa2022/blob/main/MNIST_Tarefa_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reconhecimento de dígitos da MNIST Tarefa 2
## *Paula Torales Leite*
### Case: MNIST

Usando a base MNIST, devo apresentar a classificação dela utilizando CNN.
Esse Notebook foi duplicado da Tarefa 1 de MNIST e será expandido com as demandas da presente tarefa.
Além disso, será utilizado o Modelo disponibilizado para construir o CNN.
Logo após, os resultados da acurácia do exercício anterior, serão comparados com o CNN.

## Importações

In [2]:
# Python
import sys
assert sys.version_info >= (3, 7)

# Bibliotecas padrões
import numpy as np
import random as rn
import pandas as pd

# Bibliotecas de Visualização
from matplotlib import pyplot
from sklearn.model_selection import train_test_split

# Modelagem e Aprendizagem de Máquina
from packaging import version
import sklearn
assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron

import tensorflow as tf
assert version.parse(tf.__version__) >= version.parse("2.8.0")

In [3]:
# GPU
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. Neural nets can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware "
              "accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

In [4]:
# Importando a base MNIST
from sklearn.datasets import fetch_openml
mnist = fetch_openml(name = 'mnist_784')

## Preparação dos Dados

Criação de variáveis X e y

In [5]:
X = mnist['data']
y = mnist['target']

Normalização dos dados

In [6]:
y = y.astype('int32')

X = X/ 255.0

Treinamento

In [ ]:
SEED=21

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=SEED)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.2, random_state=SEED)

print ("Dados de Treinamento: ", X_train.shape, y_train.shape)
print ("Dados de Teste: ", X_test.shape, y_test.shape)
print ("Dados de Validação: ", X_val.shape, y_val.shape)

Dados de Treinamento:  (44800, 784) (44800,)
Dados de Teste:  (14000, 784) (14000,)
Dados de Validação:  (11200, 784) (11200,)


## Árvore de Decisão

Treinamento do modelo da árvore de decisão sem podar.

In [ ]:
arvore_decisao = DecisionTreeClassifier(max_depth=None)
arvore_decisao.fit(X_train, y_train)

DecisionTreeClassifier()

Acurácia de Treinamento e Desenvolvimento

In [ ]:
print("Na profundidade:", arvore_decisao.tree_.max_depth)
print("Acurácia da Árvore de Decisão nos dados de treinamento: ", arvore_decisao.score(X_train, y_train))
print("Acurácia da Árvore de Decisão nos dados de Dev:", arvore_decisao.score(X_val, y_val))

Na profundidade: 45
Acurácia da Árvore de Decisão nos dados de treinamento:  1.0
Acurácia da Árvore de Decisão nos dados de Dev: 0.8667857142857143


Construção do modelo levando em consideração a melhor profundidade

In [ ]:
profundidade = range(2, 15)

melhor_resultado_treinamento = []
melhor_resultado_dev = []

for d in profundidade:
  clf = DecisionTreeClassifier(max_depth=d)
  clf.fit(X_train, y_train)
    
  resultado = clf.score(X_train, y_train)
  melhor_resultado_treinamento.append(resultado)
    
  resultado = clf.score(X_val, y_val)
  melhor_resultado_dev.append(resultado)

Identificação da melhor profundidade

In [ ]:
melhor_profundidade = profundidade[np.argmax(melhor_resultado_dev)]

resultado_teste = []

melhor_arvore_decisao = DecisionTreeClassifier(max_depth=melhor_profundidade)
melhor_arvore_decisao.fit(X, y)

DecisionTreeClassifier(max_depth=14)

Predições

In [ ]:
y_pred = arvore_decisao.predict(X_test)

resultado = arvore_decisao.score(X_test, y_pred)
resultado_teste = []
resultado_teste.append(arvore_decisao)

Acurácia Final

In [ ]:
print("Acurácia da Árvore de Decisão nos dados de teste: ", melhor_arvore_decisao.score(X_test, y_pred))

Acurácia da Árvore de Decisão nos dados de teste:  0.8652142857142857


Matriz de Confusão

In [ ]:
classes = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

conf_mat = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(conf_mat, columns=classes, index=classes)
cm_df

,1,2,3,4,5,6,7,8,9,10
1,1263,2,14,17,4,23,23,7,16,5
2,0,1506,9,11,8,7,7,13,16,7
3,23,15,1183,23,13,16,26,38,40,12
4,9,10,52,1157,10,54,13,25,63,35
5,8,7,15,14,1176,13,19,25,20,64
6,18,16,14,56,25,1026,40,9,42,29
7,18,10,20,6,21,27,1262,6,26,5
8,3,12,24,10,24,6,6,1309,12,41
9,16,24,49,46,27,41,32,17,1075,39
10,5,3,21,33,67,15,13,37,34,1147


## kNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3).fit(X, y)

Predição

In [ ]:
y_pred = knn.predict(X_test)

Acurácia

In [ ]:
print("Acurácia da Árvore do kNN nos dados de teste: ", knn.score(X_test, y_pred))

Acurácia da Árvore do kNN nos dados de teste:  1.0


Matriz de Confusão

In [ ]:
classes = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

conf_mat = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(conf_mat, columns=classes, index=classes)
cm_df

,1,2,3,4,5,6,7,8,9,10
1,1369,1,1,0,0,0,1,1,0,1
2,0,1583,1,0,0,0,0,0,0,0
3,4,1,1372,2,0,0,1,7,2,0
4,0,1,6,1408,0,3,0,4,1,5
5,0,7,0,0,1345,0,3,2,0,4
6,4,1,1,4,2,1251,6,1,0,5
7,4,1,0,0,0,1,1395,0,0,0
8,0,10,3,0,0,0,0,1428,1,5
9,3,10,1,12,4,11,3,5,1308,9
10,3,1,0,6,5,3,0,7,1,1349


## Perceptron

Construindo Modelo

In [ ]:
perceptron = Perceptron(tol=1e-3, random_state=0)
perceptron.fit(X_train, y_train)

Perceptron()

Acurácia

In [ ]:
perceptron.score(X, y)

0.7826142857142857

Predição

In [ ]:
y_pred = perceptron.predict(X_test)

Árvore de Decisão

In [ ]:
classes = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

conf_mat = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(conf_mat, columns=classes, index=classes)
cm_df

,1,2,3,4,5,6,7,8,9,10
1,1211,8,6,1,0,111,5,13,19,0
2,0,1494,3,0,0,10,0,5,71,1
3,7,166,987,20,3,41,5,69,86,5
4,4,110,21,1019,0,151,2,40,75,6
5,1,214,8,0,770,115,3,109,84,57
6,3,41,2,30,0,1067,3,22,100,7
7,11,111,11,0,0,126,1090,16,36,0
8,3,41,4,0,1,8,0,1374,9,7
9,2,99,3,15,0,69,0,15,1160,3
10,7,89,0,12,1,79,0,268,73,846


## CNN

A implementação desse CNN foi feita com base nessa implementação: https://github.com/ageron/handson-ml3/blob/main/14_deep_computer_vision_with_cnns.ipynb.
Dentro dela, utilizou-se o sub-tópico "CNN Architectures".

In [13]:
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_train_full = np.expand_dims(X_train_full, axis=-1).astype(np.float32) / 255
X_test = np.expand_dims(X_test.astype(np.float32), axis=-1) / 255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

In [14]:
from functools import partial

tf.random.set_seed(42)  # extra code – ensures reproducibility
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, padding="same",
                        activation="relu", kernel_initializer="he_normal")
model = tf.keras.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPool2D(),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=64, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=10, activation="softmax")
])

In [15]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))
score = model.evaluate(X_test, y_test)
X_new = X_test[:10]  # pretend we have new images
y_pred = model.predict(X_new)

Epoch 1/10
1719/1719 [==============================] - 19s 10ms/step - loss: 0.7081 - accuracy: 0.7567 - val_loss: 0.3329 - val_accuracy: 0.8778
Epoch 2/10
1719/1719 [==============================] - 17s 10ms/step - loss: 0.3964 - accuracy: 0.8677 - val_loss: 0.2955 - val_accuracy: 0.8938
Epoch 3/10
1719/1719 [==============================] - 15s 9ms/step - loss: 0.3319 - accuracy: 0.8895 - val_loss: 0.2764 - val_accuracy: 0.8994
Epoch 4/10
1719/1719 [==============================] - 16s 9ms/step - loss: 0.3048 - accuracy: 0.8963 - val_loss: 0.2540 - val_accuracy: 0.9012
Epoch 5/10
1719/1719 [==============================] - 15s 9ms/step - loss: 0.2756 - accuracy: 0.9063 - val_loss: 0.2532 - val_accuracy: 0.9110
Epoch 6/10
1719/1719 [==============================] - 14s 8ms/step - loss: 0.2588 - accuracy: 0.9115 - val_loss: 0.2548 - val_accuracy: 0.9092
Epoch 7/10
1719/1719 [==============================] - 14s 8ms/step - loss: 0.2459 - accuracy: 0.9172 - val_loss: 0.2662 - val_

## Análise

Como foi possível verificar anteriormente, as acurácias dos modelos são da seguinte forma:

*   Árvore de Decisão: 0,8652142857142857
*   kNN: 1
*   Perceptron: 0,7826142857142857
*   CNN: 0,9095

Sendo assim, é possível verificar que os dois melhores resultados foram obtidos a partir do kNN e do CNN, seguidos da Árvore de Decisão e o Perceptron.


## Bibliografia

https://github.com/Aveshnee/MNIST-Classification/blob/main/MNIST%20Logistic%20Regression%20%26%20Decision%20Trees.ipynb

https://www.youtube.com/watch?v=ooQtUaCExa8&t=721s

https://github.com/ageron/handson-ml3/blob/main/14_deep_computer_vision_with_cnns.ipynb